In [2]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sklearn.metrics       as Metrics
import pandas                as pd
import matplotlib.pyplot     as plt


import folium  #needed for interactive map
from folium.plugins import HeatMap

from   collections           import Counter
from   sklearn               import preprocessing
from   datetime              import datetime
from   collections           import Counter
from   math                  import exp
from   sklearn.linear_model  import LinearRegression as LinReg
from   sklearn.metrics       import mean_absolute_error
from   sklearn.metrics       import median_absolute_error
from   sklearn.metrics       import r2_score

%matplotlib inline
sns.set()

In [3]:
df = pd.read_csv('datasets/listings.csv', delimiter=',')
df.shape

(59824, 29)

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
df.head()

,accommodates,amenities,availability_30,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,has_availability,host_id,id,instant_bookable,latitude,longitude,metropolitan,name,price,property_type,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,state,weekly_price,zipcode
0,2.0,"{""Cable TV"",""Wireless Internet"",""Air condition...",24,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,119445,7949480,f,40.852054,-73.788680,NYC,City Island Sanctuary relaxing BR & Bath w Par...,99.0,House,10.0,10.0,10.0,10.0,100.0,10.0,Private room,NY,NaN,10464
1,4.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",30,1.0,Real Bed,1.0,1.0,flexible,sunnysidebronx,NaN,9117975,16042478,t,40.853491,-73.788607,NYC,WATERFRONT STUDIO APARTMENT,200.0,Apartment,NaN,NaN,NaN,NaN,NaN,NaN,Private room,NY,NaN,10464
2,4.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,3.0,Real Bed,3.0,3.0,strict,sunnysidebronx,NaN,9815788,1886820,f,40.841144,-73.783052,NYC,Quaint City Island Community.,300.0,House,NaN,NaN,NaN,NaN,NaN,NaN,Entire home/apt,NY,NaN,10464
3,3.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",8,1.0,Real Bed,1.0,1.0,strict,long island city,NaN,13886510,6627449,f,40.849775,-73.786609,NYC,Large 1 BDRM in Great location,125.0,Apartment,10.0,10.0,10.0,10.0,93.0,10.0,Entire home/apt,NY,775.0,10464
4,4.0,"{Internet,""Wireless Internet"",""Air conditionin...",17,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,28811542,5557381,t,40.850024,-73.789328,NYC,Quaint City Island Home,69.0,House,10.0,10.0,10.0,10.0,97.0,10.0,Private room,NY,350.0,10464


In [5]:
df.count()

accommodates                   59763
amenities                      59824
availability_30                59824
bathrooms                      59622
bed_type                       59824
bedrooms                       59729
beds                           59715
cancellation_policy            59824
city                           59778
has_availability                   0
host_id                        59824
id                             59824
instant_bookable               59824
latitude                       59824
longitude                      59824
metropolitan                   59824
name                           59783
price                          59824
property_type                  59820
review_scores_checkin          45478
review_scores_cleanliness      45544
review_scores_communication    45539
review_scores_location         45475
review_scores_rating           45624
review_scores_value            45470
room_type                      59824
state                          59824
w

In [7]:
df.dtypes

accommodates                   float64
amenities                       object
availability_30                  int64
bathrooms                      float64
bed_type                        object
bedrooms                       float64
beds                           float64
cancellation_policy             object
city                            object
has_availability               float64
host_id                          int64
id                               int64
instant_bookable                object
latitude                       float64
longitude                      float64
metropolitan                    object
name                            object
price                          float64
property_type                   object
review_scores_checkin          float64
review_scores_cleanliness      float64
review_scores_communication    float64
review_scores_location         float64
review_scores_rating           float64
review_scores_value            float64
room_type                

In [11]:
NewListings = pd.DataFrame()

In [17]:
NewListings['ID_House'] = df['id']
NewListings['Property_type'] = df['property_type']
NewListings['Room_type'] = df['room_type']
NewListings['Price'] = df['price']
NewListings['People_Cap'] = df['accommodates']
NewListings['Amenities'] = df['amenities']
NewListings['Bathrooms'] = df['bathrooms']
NewListings['Bedrooms'] = df['bedrooms']
NewListings['Name'] = df['name']
NewListings['City'] = df['city']
NewListings['Metropolitan_Area'] = df['metropolitan']
NewListings['State'] = df['state']
NewListings['Latitude'] = df['latitude']
NewListings['Longitude'] = df['longitude']
NewListings['Zipcode'] = df['zipcode']

In [18]:
NewListings.head()

,ID_House,Property_type,Room_type,Price,People_Cap,Amenities,Bathrooms,Bedrooms,Name,City,Metropolitan_Area,State,Latitude,Longitude,Zipcode
0,7949480,House,Private room,99.0,2.0,"{""Cable TV"",""Wireless Internet"",""Air condition...",1.0,1.0,City Island Sanctuary relaxing BR & Bath w Par...,sunnysidebronx,NYC,NY,40.852054,-73.788680,10464
1,16042478,Apartment,Private room,200.0,4.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,1.0,WATERFRONT STUDIO APARTMENT,sunnysidebronx,NYC,NY,40.853491,-73.788607,10464
2,1886820,House,Entire home/apt,300.0,4.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",3.0,3.0,Quaint City Island Community.,sunnysidebronx,NYC,NY,40.841144,-73.783052,10464
3,6627449,Apartment,Entire home/apt,125.0,3.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,1.0,Large 1 BDRM in Great location,long island city,NYC,NY,40.849775,-73.786609,10464
4,5557381,House,Private room,69.0,4.0,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,Quaint City Island Home,sunnysidebronx,NYC,NY,40.850024,-73.789328,10464


In [19]:
NewListings.shape

(59824, 15)

In [28]:
NewListings.groupby('State').size()

State
CO     2505
DC     7755
IL     5207
MA     3585
MD       31
MP        1
NJ        1
NY    40738
VT        1
dtype: int64

In [42]:
NewListings.groupby(['Property_type'])["Room_type"].sum()

Property_type
Apartment             Private roomEntire home/aptPrivate roomEntire ...
Bed & Breakfast       Private roomPrivate roomShared roomPrivate roo...
Boat                  Entire home/aptEntire home/aptEntire home/aptE...
Boutique hotel        Private roomPrivate roomPrivate roomEntire hom...
Bungalow              Entire home/aptEntire home/aptEntire home/aptP...
Cabin                 Entire home/aptShared roomPrivate roomEntire h...
Camper/RV             Entire home/aptEntire home/aptEntire home/aptP...
Castle                Private roomEntire home/aptPrivate roomEntire ...
Cave                                     Entire home/aptEntire home/apt
Chalet                                                     Private room
Condominium           Entire home/aptEntire home/aptEntire home/aptP...
Dorm                  Private roomPrivate roomShared roomPrivate roo...
Earth House                     Private roomPrivate roomEntire home/apt
Entire Floor          Entire home/aptPrivate roomE